# Face Mask Detection Training

Importing important libraries. Proceed if you have the following modules installed already, otherwise:

```batch
py -m pip install numpy opencv-python keras sklearn tensorflow imutils
```

In [1]:
import cv2
import imutils
import h5py
import numpy as np
import warnings

from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

warnings.filterwarnings(action='ignore', category=FutureWarning)

This convolutional network utilizes a combination of two Convolutional and MaxPooling layers to extract features from the dataset. Subsequently, a Flatten layer is employed to convert the data into a one-dimensional format, followed by a Dropout layer to mitigate overfitting.

Finally, the network incorporates two Dense layers for classification purposes.

In [2]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),

    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dropout(0.5),

    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

The generation and augmentation of the image data for training and validation.

In [3]:
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory('../dataset/train', batch_size=10, target_size=(150, 150))

validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory('../dataset/test', batch_size=10, target_size=(150, 150))

Found 1315 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


Initialize a callback checkpoint that saves the best model after each epoch during training.

In [4]:
checkpoint = ModelCheckpoint('../models/epoch/model-{epoch:03d}.model',
                             monitor='val_loss',
                             verbose=0,
                             save_best_only=True,
                             mode='auto')

Actual model training and fit generation.

In [5]:
history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Epoch 1/10


C:\Users\natha\AppData\Local\Temp\ipykernel_10996\3620006576.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


132/132 [==============================] - ETA: 0s - loss: 0.5400 - acc: 0.7323INFO:tensorflow:Assets written to: ../models/epoch\model-001.model\assets


INFO:tensorflow:Assets written to: ../models/epoch\model-001.model\assets


132/132 [==============================] - 66s 489ms/step - loss: 0.5400 - acc: 0.7323 - val_loss: 0.1609 - val_acc: 0.9381
Epoch 2/10
132/132 [==============================] - 63s 474ms/step - loss: 0.3170 - acc: 0.8677 - val_loss: 0.2330 - val_acc: 0.9433
Epoch 3/10
132/132 [==============================] - ETA: 0s - loss: 0.3280 - acc: 0.8814INFO:tensorflow:Assets written to: ../models/epoch\model-003.model\assets


...

132/132 [==============================] - 64s 483ms/step - loss: 0.3280 - acc: 0.8814 - val_loss: 0.1607 - val_acc: 0.9330
Epoch 4/10
132/132 [==============================] - ETA: 0s - loss: 0.2501 - acc: 0.9087INFO:tensorflow:Assets written to: ../models/epoch\model-004.model\assets


INFO:tensorflow:Assets written to: ../models/epoch\model-004.model\assets


132/132 [==============================] - 63s 477ms/step - loss: 0.2501 - acc: 0.9087 - val_loss: 0.0911 - val_acc: 0.9639
Epoch 5/10
132/132 [==============================] - 63s 472ms/step - loss: 0.2144 - acc: 0.9194 - val_loss: 0.1719 - val_acc: 0.9278
Epoch 6/10
132/132 [==============================] - ETA: 0s - loss: 0.2201 - acc: 0.9110INFO:tensorflow:Assets written to: ../models/epoch\model-006.model\assets


INFO:tensorflow:Assets written to: ../models/epoch\model-006.model\assets


132/132 [==============================] - 62s 467ms/step - loss: 0.2201 - acc: 0.9110 - val_loss: 0.0701 - val_acc: 0.9742
Epoch 7/10
132/132 [==============================] - 61s 463ms/step - loss: 0.2078 - acc: 0.9247 - val_loss: 0.0772 - val_acc: 0.9742
Epoch 8/10
132/132 [==============================] - 61s 461ms/step - loss: 0.1915 - acc: 0.9270 - val_loss: 0.0913 - val_acc: 0.9691
Epoch 9/10
132/132 [==============================] - 61s 461ms/step - loss: 0.1783 - acc: 0.9323 - val_loss: 0.1149 - val_acc: 0.9588
Epoch 10/10
132/132 [==============================] - 62s 468ms/step - loss: 0.1959 - acc: 0.9331 - val_loss: 0.1138 - val_acc: 0.9485


Now, save the model to a file that will be loaded later.

In [6]:
model.save("../models/keras_model/maskup-model.keras")